In [ ]:
# Se sei in notebook:
# !pip install -q "transformers>=4.42.0" "datasets>=2.19.0" "accelerate>=0.31.0" "trl>=0.9.6" "peft>=0.11.1" bitsandbytes sentencepiece evaluate
#
# Se sei in .py, installa da terminale prima di lanciare lo script.


In [11]:
# pip install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio

In [26]:
import platform
print(platform.architecture())


('64bit', 'WindowsPE')


In [1]:
import torch
print("Torch:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)  # None = build CPU-only


Torch: 2.6.0+cu124
torch.version.cuda: 12.4


In [2]:
import os, random, torch
from pathlib import Path

# Riproducibilità
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

# Rileva GPU e imposta dtype
HAS_CUDA = torch.cuda.is_available()
BF16_OK = HAS_CUDA and torch.cuda.get_device_properties(0).total_memory > 20e9
DTYPE = torch.bfloat16 if BF16_OK else torch.float16

print(f"CUDA: {HAS_CUDA}  |  BF16: {BF16_OK}  |  DTYPE: {DTYPE}")

# Cartella output per modelli e log
OUT_DIR = Path("outputs/sft_lora_tinyllama")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Disattiva la parallelizzazione dei tokenizer (per log più puliti)
os.environ["TOKENIZERS_PARALLELISM"] = "false"


CUDA: True  |  BF16: False  |  DTYPE: torch.float16


In [3]:
import sys, torch, os
print("Python:", sys.executable)
print("Torch:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)   # deve mostrare "12.x", NON None
print("is_available:", torch.cuda.is_available())
print("device_count:", torch.cuda.device_count())
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Python: c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Scripts\python.exe
Torch: 2.5.1+cu121
torch.version.cuda: 12.1
is_available: True
device_count: 1
CUDA_VISIBLE_DEVICES: None
GPU: NVIDIA RTX A3000 12GB Laptop GPU


### Import, seed e configurazione base

In [ ]:
import os, random, torch
from pathlib import Path

# Riproducibilità
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

# Rileva GPU e imposta dtype
HAS_CUDA = torch.cuda.is_available()
BF16_OK = HAS_CUDA and torch.cuda.get_device_properties(0).total_memory > 20e9
DTYPE = torch.bfloat16 if BF16_OK else torch.float16

print(f"CUDA: {HAS_CUDA}  |  BF16: {BF16_OK}  |  DTYPE: {DTYPE}")

# Cartella output per modelli e log
OUT_DIR = Path("outputs/sft_lora_tinyllama")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Disattiva la parallelizzazione dei tokenizer (per log più puliti)
os.environ["TOKENIZERS_PARALLELISM"] = "false"


CUDA: True  |  BF16: False  |  DTYPE: torch.float16


### Scelta modello e tokenizer

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Modello piccolo "chatty" compatibile con LoRA/QLoRA
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Carica tokenizer (fast) e definisci il pad_token se assente
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True, padding_side="right")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Special tokens:", tokenizer.special_tokens_map)


c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\laudi\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need t

Special tokens: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}


'</s>'

### Caricamento dataset (Databricks Dolly 15k) e formattazione

In [ ]:
from datasets import load_dataset

# Carica l'intero dataset (train/test/val non sempre presenti: Dolly ha 'train' e a volte 'test')
# https://huggingface.co/datasets/databricks/databricks-dolly-15k
raw_ds = load_dataset("databricks/databricks-dolly-15k")
print(raw_ds)

# Dolly ha campi tipici: instruction, context, response
# Creiamo un formato testuale unico per l'SFT con un semplice "template" stile chat.
SYSTEM = "Sei un assistente utile, educato e competente."

def format_record(ex):
    instr = ex.get("instruction", "") or ""
    ctx   = ex.get("context", "") or ""
    resp  = ex.get("response", "") or ""
    if ctx.strip():
        prompt = (
            f"<s>[SYSTEM]\n{SYSTEM}\n[/SYSTEM]\n"
            f"[USER]\n{instr}\nContesto: {ctx}\n[/USER]\n[ASSISTANT]\n"
        )
    else:
        prompt = (
            f"<s>[SYSTEM]\n{SYSTEM}\n[/SYSTEM]\n"
            f"[USER]\n{instr}\n[/USER]\n[ASSISTANT]\n"
        )
    text = prompt + resp  # per causal LM mettiamo prompt + risposta come unico testo
    return {"text": text}

# Applica il formatter e rimuovi le colonne originali
ds = raw_ds.map(format_record, remove_columns=raw_ds["train"].column_names)
print(ds)
print("Esempio:", ds["train"][0]["text"][:400], "...")


c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\laudi\.cache\huggingface\hub\datasets--databricks--databricks-dolly-15k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' packag

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})


Map: 100%|██████████| 15011/15011 [00:02<00:00, 5066.78 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15011
    })
})
Esempio: <s>[SYSTEM]
Sei un assistente utile, educato e competente.
[/SYSTEM]
[USER]
When did Virgin Australia start operating?
Contesto: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It sudde ...


### (Opzionale) QLoRA 4-bit: quantizzazione per ridurre VRAM

In [6]:
from transformers import BitsAndBytesConfig

# Config per caricare il modello in 4-bit (QLoRA). Richiede GPU + bitsandbytes.
# Se NON hai GPU, SALTA questa sezione e carica il modello "normale" in STEP 5.
bnb_config = None
if HAS_CUDA:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,                 # attiva quantizzazione 4-bit
        bnb_4bit_quant_type="nf4",         # quantizzazione "normal float 4"
        bnb_4bit_use_double_quant=True,    # double quant per meno memoria
        bnb_4bit_compute_dtype=DTYPE       # compute in bf16/fp16
    )


### Carica il modello e “avvolgilo” con LoRA (PEFT)

In [28]:
from peft import LoraConfig, get_peft_model, TaskType

# Carica il modello base: con quantizzazione se disponibile, altrimenti in fp16/bf16 quindi usiamo QLoRa
if bnb_config is not None:
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        device_map="auto"
    )
else:
    # CPU o GPU senza quantizzazione: attenzione ai consumi
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=DTYPE,
        device_map="auto" if HAS_CUDA else None
    )

# Configurazione LoRA:
# r: rank delle matrici low-rank; lora_alpha: scaling; lora_dropout: regolarizzazione
# target_modules: i sotto-layer su cui applicare LoRA (tipico per famiglie LLaMA)
lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"]  # puoi includere anche gate/up/down_proj
)

# Avvolgi il modello con PEFT-LoRA
model = get_peft_model(model, lora_cfg)

# Abilita gradient checkpointing per ridurre memoria (leggero overhead computazionale)
model.gradient_checkpointing_enable()
print("Parametri trainabili (LoRA) vs totali:")
model.print_trainable_parameters()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Parametri trainabili (LoRA) vs totali:
trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


### Trainer SFT (TRL) con collator e argomenti di training

In [ ]:
# !pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 1.4 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.7 MB 1.7 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.7 MB/s  0:00:01


In [37]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig

# Crea config SFT senza peft_config dentro
sft_cfg = SFTConfig(
    dataset_text_field="text",
    max_length=1024,
    assistant_only_loss=False,
    completion_only_loss=True,
    packing=False
)

# Crea config LoRA (PEFT)
lora_cfg = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    bias="none"
)

trainer = SFTTrainer(
    model=model,
    args=sft_cfg,
    train_dataset=ds["train"].select(range(5000)),
    peft_config=lora_cfg,
    data_collator=collator,
    processing_class=tokenizer
)

trainer.train()


c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Lib\site-packages\peft\tuners\tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Truncating train dataset: 100%|██████████| 5000/5000 [00:00<00:00, 263107.63 examples/s]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:745: 

Step,Training Loss
10,2.220200
20,2.104000
30,2.038500
40,1.905600
50,1.825600
60,1.787100
70,1.767200
80,1.649600
90,1.584700
100,1.455100


c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\laudi\OneDrive\Documenti\REPO\DeepLearning_MachineLearning\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences betwe

TrainOutput(global_step=1875, training_loss=1.4763242940266927, metrics={'train_runtime': 4684.8568, 'train_samples_per_second': 3.202, 'train_steps_per_second': 0.4, 'total_flos': 5.486324167272038e+16, 'train_loss': 1.4763242940266927, 'entropy': 1.3060927152633668, 'num_tokens': 3537546.0, 'mean_token_accuracy': 0.7046420693397522, 'epoch': 3.0})

### Salvataggio adapter LoRA e tokenizer

In [38]:
ADAPTER_DIR = OUT_DIR / "lora_adapter"
ADAPTER_DIR.mkdir(exist_ok=True, parents=True)

# Salva SOLO i pesi LoRA (pochi MB) + tokenizer
trainer.model.save_pretrained(str(ADAPTER_DIR))
tokenizer.save_pretrained(str(ADAPTER_DIR))
print("Adapter LoRA salvato#  in:", ADAPTER_DIR)


Adapter LoRA salvato#  in: outputs\sft_lora_tinyllama\lora_adapter


### Inferenzia con l’adapter (senza fare merge)

In [ ]:
from peft import PeftModel
from transformers import pipeline, AutoModelForCausalLM

# Ricarica il modello base (quantizzato se possibile) e attacca l'adapter LoRA
if bnb_config is not None:
    base = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        device_map="auto"
    )
else:
    base = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=DTYPE,
        device_map="auto" if HAS_CUDA else None
    )

ft = PeftModel.from_pretrained(base, str(ADAPTER_DIR))

# Pipeline di generazione
gen = pipeline(
    "text-generation",
    model=ft,
    tokenizer=tokenizer,
    device_map="auto"
)

prompt = (
    "<s>[SYSTEM]\nSei un assistente fiscale conciso.\n[/SYSTEM]\n"
    "[USER]\nSpiega l'IVA in Italia in 3 punti chiari.\n[/USER]\n[ASSISTANT]\n"
)
out = gen(prompt, max_new_tokens=200, do_sample=True, temperature=0.7, top_p=0.9)



Device set to use cuda:0


<s>[SYSTEM]
Sei un assistente fiscale conciso.
[/SYSTEM]
[USER]
Spiega l'IVA in Italia in 3 punti chiari.
[/USER]
[ASSISTANT]
In Italia, l'IVA è una forma di cassa fiscale che è stata istituita nel 1924. È una cassa fiscale che è stata istituita con l'intento di migliorare la regolamentazione dell'economia del paese. L'IVA è la cassa fiscale che viene utilizzata per i salari e gli imposti a cassa fiscale. L'IVA è una cassa fiscale che è stata istituita con l'intento di migliorare la regolamentazione dell'economia del paese.

In Italia, l'IVA è una forma di cassa fiscale che è stata istituita nel 1924. L'IVA è una cassa fiscale che è stata istituita con l'intento di migliorare la regolamentazione dell'


In [40]:
out

[{'generated_text': "<s>[SYSTEM]\nSei un assistente fiscale conciso.\n[/SYSTEM]\n[USER]\nSpiega l'IVA in Italia in 3 punti chiari.\n[/USER]\n[ASSISTANT]\nIn Italia, l'IVA è una forma di cassa fiscale che è stata istituita nel 1924. È una cassa fiscale che è stata istituita con l'intento di migliorare la regolamentazione dell'economia del paese. L'IVA è la cassa fiscale che viene utilizzata per i salari e gli imposti a cassa fiscale. L'IVA è una cassa fiscale che è stata istituita con l'intento di migliorare la regolamentazione dell'economia del paese.\n\nIn Italia, l'IVA è una forma di cassa fiscale che è stata istituita nel 1924. L'IVA è una cassa fiscale che è stata istituita con l'intento di migliorare la regolamentazione dell'"}]

### (Opzionale) Merge LoRA nel backbone per esportare un singolo checkpoint

In [ ]:
# ATTENZIONE: il merge richiede più memoria (temporaneamente).
merged = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=DTYPE,
    device_map="auto" if HAS_CUDA else None
)
merged = PeftModel.from_pretrained(merged, str(ADAPTER_DIR))
merged = merged.merge_and_unload()  # fonde gli adapter nei pesi del modello

MERGED_DIR = OUT_DIR / "merged_full"
MERGED_DIR.mkdir(exist_ok=True, parents=True)
merged.save_pretrained(str(MERGED_DIR))
tokenizer.save_pretrained(str(MERGED_DIR))
print("Modello fuso salvato in:", MERGED_DIR)
